In [ ]:
import requests
import pandas as pd
from itertools import product
from time import sleep

# Define filters
geo_list = ['EU27_2020','BE','BG','CZ','DK','DE','EE','IE','EL','ES','FR','HR','IT','CY','LV','LT','LU','HU','MT','NL','AT','PL','PT','RO','SI','SK','FI','SE']
wstatus_list = ['EMP','SAL','NSAL','SELF','SELF_S','SELF_NS','CFAM','NCFAM','NRP']
worktime_list = ['TOTAL','PT','FT','NRP']
age_list = ['Y15-24','Y15-34','Y15-64','Y_GE15','Y20-64','Y25-54','Y25-64','Y35-49','Y_GE50','Y55-64']
sex_list = ['T','M','F']


base_url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/lfsa_ewhun2"
common_params = {
    'format': 'JSON',
    'unit': 'HR',
    'nace_r2': 'TOTAL',
    'lang': 'en'
}

results = []


for geo in geo_list:
    for (wstatus, worktime, age, sex) in product(wstatus_list, worktime_list, age_list, sex_list):
        params = common_params.copy()
        params.update({
            'geo': geo,
            'wstatus': wstatus,
            'worktime': worktime,
            'age': age,
            'sex': sex
        })
        try:
            response = requests.get(base_url, params=params, timeout=20)
            response.raise_for_status()
            data = response.json()

            if "value" not in data or "dimension" not in data:
                continue  


            time_labels = data['dimension']['time']['category']['label']
            geo_name = data['dimension']['geo']['category']['label'][geo]

            for key, val in data.get('value', {}).items():
                time_idx = int(key) % len(time_labels)
                year = list(time_labels.values())[time_idx]
                results.append({
                    'country': geo_name,
                    'wstatus': wstatus,
                    'worktime': worktime,
                    'age': age,
                    'sex': sex,
                    'year': year,
                    'hours_per_week': val
                })
        except Exception as e:
            print(f"Error for {geo} {wstatus} {worktime} {age} {sex}: {e}")
        sleep(0.3)  

# save
df = pd.DataFrame(results)
df.to_csv("workhours_data.csv", index=False)
print("Saved to workhours_data.csv")
